# (Toy) Fortran Frontend for Loopy

In [10]:
import loopy as lp

In [3]:
src = """subroutine sparse(rowstarts, colindices, values, m, n, nvals, x, y)
  implicit none

  integer rowstarts(m+1), colindices(nvals)
  real*8 values(nvals)
  real*8 x(n), y(n), rowsum

  integer m, n, rowstart, rowend, length, nvals
  integer i, j

  do i = 1, m
    rowstart = rowstarts(i)
    rowend = rowstarts(i+1)
    length = rowend - rowstart

    rowsum = 0
    do j = 1, length
      rowsum = rowsum + &
        x(colindices(rowstart+j-1))*values(rowstart+j-1)
    end do
    y(i) = rowsum
  end do
end
"""

`lp.parse_fortran(src, filename)`

In [14]:
sparse_t = lp.split_iname(sparse, "i", 128)
sparse_t = lp.tag_inames(sparse_t, {"i_outer": "g.0"})
sparse_t = lp.tag_inames(sparse_t, {"i_inner": "l.0"})
sparse_t = lp.split_iname(sparse_t, "j", 4)
sparse_t = lp.tag_inames(sparse_t, {"j_inner": "unr"})
print(sparse_t)

Now generate code:

In [15]:
print(lp.generate_code_v2(sparse_t).device_code())